In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors

### Reading files

In [2]:
df = pd.read_csv('data.csv')



### Understanding The Data and Cleaning it

In [3]:
df.head()
df_meeting_data = df.iloc[:1]
df_meeting_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Meeting ID          1 non-null      object
 1   Topic               1 non-null      object
 2   Start Time          1 non-null      object
 3   End Time            1 non-null      object
 4   User Email          1 non-null      object
 5   Duration (Minutes)  1 non-null      object
 6   Participants        1 non-null      object
 7   Unnamed: 7          0 non-null      object
dtypes: object(8)
memory usage: 192.0+ bytes


In [4]:
meeting_duration = df_meeting_data["Duration (Minutes)"]
meeting_duration = int(meeting_duration)
df_participants = df.iloc[1:].rename(columns=df.iloc[1]).drop(index=1)
df_participants.head()
df_participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 2 to 65
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Name (Original Name)  64 non-null     object
 1   User Email            42 non-null     object
 2   Join Time             64 non-null     object
 3   Leave Time            64 non-null     object
 4   Duration (Minutes)    64 non-null     object
 5   Guest                 64 non-null     object
 6   Recording Consent     12 non-null     object
 7   In Waiting Room       64 non-null     object
dtypes: object(8)
memory usage: 4.1+ KB


In [5]:
df_participants.columns

Index(['Name (Original Name)', 'User Email', 'Join Time', 'Leave Time',
       'Duration (Minutes)', 'Guest', 'Recording Consent', 'In Waiting Room'],
      dtype='object')

### Analyzing Students Attendance

Specifications:
- student is considered present if they attend the meeting on time or less than 10 minutes late

- student is considered late if they're late more than 20 minutes

- student is considered Absent if they were late more than 20 minutes.

In [6]:
df_participants["Student ID"] = df_participants["User Email"].str.extract(pat='(^[0-9]*)')
df_participants = df_participants.dropna(subset=['Student ID'])
df_participants["Attended Time"] = df_participants["Duration (Minutes)"].astype("int")
df_participants = df_participants.groupby("Student ID", as_index=False)["Attended Time"].sum()
df_participants["Attendance Percentage"] = (df_participants["Attended Time"]/meeting_duration)*100
df_participants["Status"] = ['Absent' if x < meeting_duration - 21 else "Late" if meeting_duration - 20 <= x < meeting_duration-10 else "Present" for x in df_participants["Attended Time"]]
df_participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Student ID             29 non-null     object 
 1   Attended Time          29 non-null     int64  
 2   Attendance Percentage  29 non-null     float64
 3   Status                 29 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.0+ KB


In [7]:
df_participants = df_participants.iloc[1:]

In [8]:
def highlight_cells(val):
    if val == 100:
        code = (0.3,0.5,1)
        color = matplotlib.colors.hsv_to_rgb(code)
        final = matplotlib.colors.to_hex(color, keep_alpha=False)
    elif val >= 50:
        code = (0.3,0.5,0.5)
        color = matplotlib.colors.hsv_to_rgb(code)
        final = matplotlib.colors.to_hex(color, keep_alpha=False)
    elif val < 50:
        code = (0.3,0.5,0.3)
        color = matplotlib.colors.hsv_to_rgb(code)
        final = matplotlib.colors.to_hex(color, keep_alpha=False)
    else:
        final = ''
    return 'background-color: {}'.format(final)

In [9]:
df_participants.style.applymap(highlight_cells, subset=['Attendance Percentage']).to_excel('data.xlsx', engine='openpyxl')